In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '19lWsG9i0fYz7mqoULzvODKyIyQLuCTVS'})
download.GetContentFile('fruits-dataset.zip')

In [ ]:
import zipfile
import io
data = zipfile.ZipFile('fruits-dataset.zip', 'r')
data.extractall()

os.rename('archive', 'fruits-dataset') # Just renaming the unzipped folder as it was zipped under the name archive

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) # the input shape here is what imagenet is trained on

# Freeze the base model layers so they are not updated during training
base_model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Prevent overfitting
x = Dense(256, activation='relu')(x)


num_classes = 33  # Change this based on your dataset

# Output layer (change the units based on your number of classes)
output_layer = Dense(num_classes, activation='softmax')(x)  # Softmax for multi-class

# Define new model
model = Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size=32 # Change to 16 or 8

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'fruits-dataset/train/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical')

# val_generator = val_datagen.flow_from_directory(
#     'path/to/val',
#     target_size=(224, 224),
#     batch_size=batch_size,
#     class_mode='categorical')

# model.fit(train_generator, validation_data=val_generator, epochs=10)
model.fit(train_generator, epochs=10)

Found 16854 images belonging to 33 classes.
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


527/527 ━━━━━━━━━━━━━━━━━━━━ 64s 92ms/step - accuracy: 0.0475 - loss: 3.5391
Epoch 2/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step - accuracy: 0.1161 - loss: 3.2765
Epoch 3/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 83s 86ms/step - accuracy: 0.2075 - loss: 2.8193
Epoch 4/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step - accuracy: 0.3293 - loss: 2.3296
Epoch 5/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step - accuracy: 0.4410 - loss: 1.9538
Epoch 6/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 86ms/step - accuracy: 0.5190 - loss: 1.6688
Epoch 7/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step - accuracy: 0.5829 - loss: 1.4366
Epoch 8/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.6328 - loss: 1.2524
Epoch 9/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 85ms/step - accuracy: 0.6714 - loss: 1.1287
Epoch 10/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 45s 86ms/step - accuracy: 0.6961 - loss: 1.0177


In [ ]:
# Save the full model
model.save("fruit_classification_model_transfer_training.keras")

In [ ]:
class_indices = train_generator.class_indices  # Dictionary mapping class names to indices
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping
index_to_class

{0: 'Apple Braeburn',
 1: 'Apple Granny Smith',
 2: 'Apricot',
 3: 'Avocado',
 4: 'Banana',
 5: 'Blueberry',
 6: 'Cactus fruit',
 7: 'Cantaloupe',
 8: 'Cherry',
 9: 'Clementine',
 10: 'Corn',
 11: 'Cucumber Ripe',
 12: 'Grape Blue',
 13: 'Kiwi',
 14: 'Lemon',
 15: 'Limes',
 16: 'Mango',
 17: 'Onion White',
 18: 'Orange',
 19: 'Papaya',
 20: 'Passion Fruit',
 21: 'Peach',
 22: 'Pear',
 23: 'Pepper Green',
 24: 'Pepper Red',
 25: 'Pineapple',
 26: 'Plum',
 27: 'Pomegranate',
 28: 'Potato Red',
 29: 'Raspberry',
 30: 'Strawberry',
 31: 'Tomato',
 32: 'Watermelon'}

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_fruit(image_path, model, index_to_class, image_size=224):
    img = image.load_img(image_path, target_size=(image_size, image_size))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    predicted_class_name = index_to_class[predicted_class_index]

    return predicted_class_name

# Example usage:
# image_path = "path_to_test_image.jpg"
# print("Predicted fruit:", predict_fruit(image_path, model, index_to_class))

In [ ]:
train_dir = 'fruits-dataset/train/train'
# validation_dir = 'fruits-dataset/validate/validate'
test_dir = 'fruits-dataset/test/test'

In [ ]:
import pandas as pd
import os

# print(os.listdir(test_dir))
max_count = 100
count = 0
results = []
for fruit_file in os.listdir(test_dir):
    if count > max_count:
        break
    if fruit_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        count += 1
        image_path = os.path.join(test_dir, fruit_file)
        # image_path = os.path.join(fruit_folder, image_file)
        predicted_class = predict_fruit(image_path, model, index_to_class)
        tempResult = {"image": fruit_file, "actual_class": image_path, "predicted_class": predicted_class}
        print(tempResult)
        results.append(tempResult)

# Convert results to a DataFrame for analysis
df_results = pd.DataFrame(results)
print(df_results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
{'image': '0307.jpg', 'actual_class': 'fruits-dataset/test/test/0307.jpg', 'predicted_class': 'Pineapple'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
{'image': '4921.jpg', 'actual_class': 'fruits-dataset/test/test/4921.jpg', 'predicted_class': 'Papaya'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
{'image': '2181.jpg', 'actual_class': 'fruits-dataset/test/test/2181.jpg', 'predicted_class': 'Pineapple'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
{'image': '0870.jpg', 'actual_class': 'fruits-dataset/test/test/0870.jpg', 'predicted_class': 'Apple Granny Smith'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
{'image': '3958.jpg', 'actual_class': 'fruits-dataset/test/test/3958.jpg', 'predicted_class': 'Onion White'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
{'image': '4929.jpg', 'actual_class': 'fruits-dataset/test/test/4929.jpg', 'predicted_class': 'Blueberry'}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
{'image': '4370.jpg', 'actual_class': 'fruits-dataset/test/test/4370.jpg', 'predicted_